In [25]:
import pandas as pd
import re
from pathlib import Path
from rapidfuzz import process, fuzz

# === Load your movie title list (deduplicated) ===
csv_path = "/Users/cedricroetheli/Desktop/Deduplicated_Movie_List.csv"
df = pd.read_csv(csv_path)
df = df.sort_values("Year").drop_duplicates(subset=["Title"], keep="first").reset_index(drop=True)

# === Extract full list of subtitle .json files ===
subs_path = Path("/Users/cedricroetheli/Desktop/processed_subs")  # Update to your actual path
subtitle_files = list(subs_path.glob("*.json"))

# === Clean subtitle filenames ===
subtitle_file_data = []

for f in subtitle_files:
    original_name = f.stem
    prefix = original_name[:5]  # year + underscore
    cleaned = re.sub(r"\d{3,4}p|BluRay|WEBRip|x264|XviD|DTS|AC3|HDRip|HEVC|BRRip|DVDRip|Atmos|\.?eng?|\.?EN|\.?SPARKS|\.?PLAYNOW|\.?WiKi|\.?LCHD|\.?CiNEFiLE", "", original_name, flags=re.IGNORECASE)
    cleaned = re.sub(r"[._]", " ", cleaned)
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    cleaned_title = f"{prefix}{cleaned[5:]}"
    subtitle_file_data.append((original_name, cleaned_title))

# === Match subtitles to movies by fuzzy title matching ===
movie_records = [(str(row["Year"]), row["Title"]) for _, row in df.iterrows()]
matches = []

for original, cleaned in subtitle_file_data:
    year_prefix = cleaned[:4]
    candidates = [t for t in movie_records if t[0] == year_prefix]
    if not candidates:
        matches.append((original, cleaned, None, 0))
        continue

    candidate_titles = [t[1] for t in candidates]
    match, score, idx = process.extractOne(cleaned[5:], candidate_titles, scorer=fuzz.token_sort_ratio)
    matches.append((original, cleaned, match, score))

# === Turn into DataFrame and inspect ===
match_df = pd.DataFrame(matches, columns=["subtitle_filename", "cleaned_name", "matched_title", "score"])
match_df = match_df.sort_values("score", ascending=False)

# Show top matches
match_df.head(10)


,subtitle_filename,cleaned_name,matched_title,score
299,2017_Logan,2017_Logan,Logan,100.0
276,1979_10,1979_10,10,100.0
427,1983_Tootsie,1983_Tootsie,Tootsie,100.0
29,2017_It,2017_It,It,100.0
125,1981_Excalibur,1981_Excalibur,Excalibur,100.0
414,1986_Top Gun,1986_Top Gun,Top Gun,100.0
123,1996_Ransom_EN,1996_Ransom,Ransom,100.0
108,1996_The First Wives Club,1996_The First Wives Club,The First Wives Club,100.0
407,1980_Stir Crazy,1980_Stir Crazy,Stir Crazy,100.0
328,2016_Deadpool,2016_Deadpool,Deadpool,100.0


In [17]:
match_df[match_df["score"] < 80].sort_values("score")




,subtitle_filename,cleaned_name,matched_title,score
61,2010_dmd-ts3,2010_dmd-ts3,Alice in Wonderland,15.384615
409,2012_MSAR. EN2,2012_MSAR 2,Skyfall,15.384615
81,1989_ARG-LETHAL_WEAPON_2_1989__PALDVDRIP__XVID...,1989_ARG-LETHAL WEAPON 2 1989 PAL MP3 A RG,Lethal Weapon 2,19.230769
347,2000_X-Men ;[2000];[1080p];[Shadowl0rd].ENG,2000_X-M ;[2000];[];[Shadowl0rd],Gladiator,22.222222
428,1992_A LEAGUE OF THEIR OWN [1992][MP3][DVDRIP-...,1992_A LEAGUE OF THEIR OWN [1992][MP3][-M333]-...,A League of Their Own,22.857143
...,...,...,...,...
359,2004_The.Bourne.Supremacy.2004.720p.BluRay.DTS...,2004_The Bourne Supremacy 2004 -ESiR,The Bourne Supremacy,78.431373
300,1991_The.Naked.Gun.2.1.2.The.Smell.of.Fear.199...,1991_The Naked Gun 2 1 2 The Smell of Fear 199...,The Naked Gun 2½: The Smell of Fear,78.571429
161,2021_Venom.Let.There.Be.Carnage.2021.HDRip.Xvi...,2021_Vom Let There Be Carnage 2021 -EVO,Venom: Let There Be Carnage,78.688525
337,1989_Back.to.the.Future.Part.II.1989.720p.BluR...,1989_Back to the Future Part II 1989 -AMIABLE,Back to the Future Part II,78.787879


In [33]:
confident_matches = match_df[match_df["score"] >= 85].copy()
uncertain_matches = match_df[match_df["score"] < 85].copy()


In [9]:
uncertain_matches.to_csv("uncertain_subtitle_matches.csv", index=False)


In [21]:
corrected_df = pd.read_csv("corrected_matches.csv")

# Merge with confident matches
final_matches = pd.concat([confident_matches, corrected_df], ignore_index=True)


In [23]:
final_matches.to_csv("final_matches.csv", index=False)


In [29]:
import pandas as pd

corrected_df = pd.read_csv("corrected_matches.csv")

# Drop bad first row if it has headers again
if corrected_df.columns[0] != "subtitle_filename":
    corrected_df = corrected_df.iloc[1:].reset_index(drop=True)
    corrected_df.columns = ["subtitle_filename", "cleaned_name", "matched_title", "score"]


In [35]:
match_df = pd.read_csv("match_df.csv")  # if saved earlier


FileNotFoundError: [Errno 2] No such file or directory: 'match_df.csv'

In [37]:
# Align column names for merge
corrected_df = corrected_df[["subtitle_filename", "cleaned_name", "matched_title", "score"]]
confident_matches = confident_matches[["subtitle_filename", "cleaned_name", "matched_title", "score"]]


In [39]:
final_matches = pd.concat([confident_matches, corrected_df], ignore_index=True)

# Drop any rows without subtitle filenames
final_matches = final_matches[final_matches["subtitle_filename"].notna()].reset_index(drop=True)

# Optional: save it for backup
final_matches.to_csv("final_matches.csv", index=False)
